Math 5750/6880: Mathematics of Data Science \
Project 3

# 1. Fashion-MNIST image classification using sklearn

In [2]:
from tensorflow.keras.datasets import fashion_mnist
from sklearn.preprocessing import StandardScaler

# Load Fashion-MNIST
# Classes (0-9): T-shirt/top, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, Ankle boot
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train = X_train.reshape(len(X_train), -1)
X_test  = X_test.reshape(len(X_test), -1)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [3]:
import warnings
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings('ignore', category=ConvergenceWarning)

In [4]:
def run_configuration(config):
    mlp = MLPClassifier(**config)
    mlp.fit(X_train, y_train)
    y_pred = mlp.predict(X_test)
    test_item = {"config":config, "accuracy":accuracy_score(y_test, y_pred)}
    return test_item

In [5]:
import random
import math

config_ranges = {
    "hidden_layer_sizes": [(100,), (50), (10, 10)],
    "max_iter": [10],
    "alpha": [1e-3, 1e-4, 1e-5],
    "solver": ["adam", "sgd"],
    "learning_rate_init": [0.05, 0.1, 0.15],
    "activation": ['relu', 'logistic', 'tanh'],
    "verbose": [0],
    "random_state": [1]
}
numerical_keys = {"alpha", "learning_rate_init"}
all_keys = list(config_ranges.keys())
mutable_keys = [key for key in all_keys if len(config_ranges[key]) > 1]

mutant_rate = 0.25
initial_population = 5
generations = 7
births_per_generation = 5
tournament_size = 3
cull_rate = 0.2
population = []

def sort_population():
    global population
    population = sorted(population, key=lambda item: -item["accuracy"])

def add_to_population(config):
    result = run_configuration(config)
    if result not in population:
       population.insert(1, result)

def random_tournament(ignore={}):
    volunteers = [body for body in population if body != ignore]
    return max(random.sample(volunteers, min(len(volunteers), tournament_size)), key=lambda body: body["accuracy"])

def reproduce(left_body, right_body):
    new_config = left_body["config"].copy()
    for each_key in random.sample(all_keys, random.randint(0, len(all_keys))):
        new_config[each_key] = right_body["config"][each_key]
    if random.uniform(0, 1) < mutant_rate:
        mutation = random.choice(mutable_keys)
        if mutation in numerical_keys:
            new_config[mutation] *= random.uniform(0.8, 1.2)
        else:
            new_config[mutation] = random.choice([each for each in config_ranges[mutation] if each != new_config[mutation]])

    return new_config

def cull_population():
    sort_population()
    global population
    population = population[:math.ceil(len(population) * (1 - cull_rate))]

def init_population():
    for _ in range(0, initial_population):
        new_config = {}
        for each_key in config_ranges:
            new_config[each_key] = random.choice(config_ranges[each_key])
        add_to_population(new_config)

init_population()

for generation in range(0, generations + 1):
    print("generation", generation, "population", len(population), "best", max(population, key=lambda body: body["accuracy"])["accuracy"])
    for _ in range(0, births_per_generation):
        left_body = random_tournament()
        right_body = random_tournament(left_body)
        add_to_population(reproduce(left_body, right_body))
    cull_population()











generation 0 population 5 best 0.8798
generation 1 population 6 best 0.8798
generation 2 population 8 best 0.8805
generation 3 population 9 best 0.8805
generation 4 population 11 best 0.8805
generation 5 population 9 best 0.8805
generation 6 population 8 best 0.8805
generation 7 population 8 best 0.8805


In [8]:

for each_body in population: print(each_body)



{'config': {'hidden_layer_sizes': (100,), 'max_iter': 10, 'alpha': 0.001, 'solver': 'sgd', 'learning_rate_init': 0.05, 'activation': 'relu', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.8805}
{'config': {'hidden_layer_sizes': (100,), 'max_iter': 10, 'alpha': 0.001, 'solver': 'sgd', 'learning_rate_init': 0.05, 'activation': 'logistic', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.8798}
{'config': {'hidden_layer_sizes': (100,), 'max_iter': 10, 'alpha': 0.0001, 'solver': 'sgd', 'learning_rate_init': 0.05, 'activation': 'logistic', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.8797}
{'config': {'hidden_layer_sizes': (100,), 'max_iter': 10, 'alpha': 0.001, 'solver': 'sgd', 'learning_rate_init': 0.040073459596627586, 'activation': 'relu', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.8789}
{'config': {'hidden_layer_sizes': (100,), 'max_iter': 10, 'alpha': 0.0001, 'solver': 'sgd', 'learning_rate_init': 0.040073459596627586, 'activation': 'relu', 'verbose': 0, 'random_state': 1}, 'ac

# 3. Fashion-MNIST image classification  using pytorch

In [6]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
import torch
from torch.utils.data import TensorDataset, DataLoader

# Load Fashion-MNIST
# Classes (0-9): T-shirt/top, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, Ankle boot
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# scale to [0,1], add channel dimension -> (N, 1, 28, 28)
X_train = (X_train.astype("float32") / 255.0)[:, None, :, :]
X_test  = (X_test.astype("float32")  / 255.0)[:,  None, :, :]

y_train = y_train.astype(np.int64)
y_test  = y_test.astype(np.int64)

# train/val split: last 10k of train as validation
X_tr, X_val = X_train[:50000], X_train[50000:]
y_tr, y_val = y_train[:50000], y_train[50000:]

# wrap in PyTorch TensorDatasets and DataLoaders
train_ds = TensorDataset(torch.from_numpy(X_tr),  torch.from_numpy(y_tr))
val_ds   = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))
test_ds  = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=256, shuffle=False)
test_loader  = DataLoader(test_ds,  batch_size=256, shuffle=False)

In [7]:
import torch.nn as nn
import torch.optim as optim

# In colab, you should ``change runtime type'' to GPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# your code here

Using device: cuda
